In [1]:
!pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Downloading dataset from kaggle...

In [2]:
import opendatasets as od
od.download('https://www.kaggle.com/datasets/alxmamaev/flowers-recognition')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: aryan7004
Your Kaggle Key: ··········


100%|██████████| 225M/225M [00:03<00:00, 67.0MB/s]


Importing the libraries

In [3]:
import cv2
import numpy
from tensorflow import keras

In [4]:
from keras.applications.vgg16 import VGG16
from keras.layers import Dense
from keras import Sequential

Importing the model VGG16 without the full connected layers

In [5]:
base_model = VGG16(include_top = False,weights = 'imagenet',input_shape=(150,150,3))
base_model.summary()

58889256/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                              

Creating our own model by adding the output layers.

In [6]:
from keras.layers import Flatten
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(1024,activation='relu'))
model.add(Dense(5,activation='softmax'))
base_model.trainable=False
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 1024)              8389632   
                                                                 
 dense_1 (Dense)             (None, 5)                 5125      
                                                                 
Total params: 23,109,445
Trainable params: 8,394,757
Non-trainable params: 14,714,688
_________________________________________________________________


Preparing the training and validation dataset

In [7]:
from tensorflow import keras
# generators
ds = keras.utils.image_dataset_from_directory(
    directory = '/content/flowers-recognition/flowers',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    seed = 10,
    image_size=(150,150),
    validation_split = 0.1,
    subset = "both"
)
print(len(ds[0]))

Found 4317 files belonging to 5 classes.
Using 3886 files for training.
Using 431 files for validation.
122


Normalizing the images present in the dataset

In [8]:
import tensorflow
# Normalize
def process(image,label):
    image = tensorflow.cast(image/255. ,tensorflow.float32)
    return image,label

ds[0] = ds[0].map(process)
ds[1] = ds[1].map(process)

Using early stopping to increase the accuracy of the model

In [14]:
callback = tensorflow.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)


Compiling the model using SparseCategoricalCrossentropy loss function

In [12]:
model.compile(loss='SparseCategoricalCrossentropy',optimizer= 'adam',metrics=['accuracy'])

Fiting the dataset into the model

In [15]:
model.fit(x=ds[0],epochs=20,verbose = 2,validation_data=ds[1],callbacks=callback)

Epoch 1/20
122/122 - 9s - loss: 0.0305 - accuracy: 0.9949 - val_loss: 0.9399 - val_accuracy: 0.7796 - 9s/epoch - 78ms/step
Epoch 2/20
122/122 - 9s - loss: 0.0246 - accuracy: 0.9964 - val_loss: 0.8575 - val_accuracy: 0.7981 - 9s/epoch - 76ms/step
Epoch 3/20
122/122 - 9s - loss: 0.0238 - accuracy: 0.9959 - val_loss: 0.9120 - val_accuracy: 0.7912 - 9s/epoch - 77ms/step
Epoch 4/20
122/122 - 9s - loss: 0.0226 - accuracy: 0.9964 - val_loss: 0.9741 - val_accuracy: 0.7935 - 9s/epoch - 76ms/step
Epoch 5/20
122/122 - 9s - loss: 0.0184 - accuracy: 0.9969 - val_loss: 0.9875 - val_accuracy: 0.7819 - 9s/epoch - 76ms/step
Epoch 6/20
122/122 - 10s - loss: 0.0229 - accuracy: 0.9959 - val_loss: 0.9812 - val_accuracy: 0.7958 - 10s/epoch - 80ms/step


Calculating the training and test accuracy

In [27]:
train_loss, train_acc = model.evaluate(x=ds[0])
test_loss, test_acc   = model.evaluate(x=ds[1])
print("final train accuracy = {:.2f} , Test accuracy = {:.2f}".format(train_acc*100, test_acc*100))

14/14 [==============================] - 1s 64ms/step - loss: 0.9812 - accuracy: 0.7958
final train accuracy = 99.74 , Test accuracy = 79.58
